# Cyber2A Workshop Hands-on Lab: Foundation Models - Image Segmentation using Segment Anything Model 2 (SAM 2)

<div class="alert alert-block alert-info">

This notebook reuses some code segments (e.g., helper methods, imports, loading the model, etc.) from the [image predictor example](https://github.com/facebookresearch/sam2/blob/main/notebooks/image_predictor_example.ipynb) initially published in the SAM 2 source code repository.
SAM 2 source code is released under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0), and <i>Meta Platforms, Inc. and affiliates</i> hold the copyright for the [image_predictor_example.ipynb](https://github.com/facebookresearch/sam2/blob/main/notebooks/image_predictor_example.ipynb) notebook.
We have adapted and modified the image predictor example notebook to use data files from Arctic datasets and included specific activities for the Cyber2A Workshop.
</div>

## Introduction

SAM-2 is a Promptable Visual Segmentation (PVS) model trained on large-scale generic data that can predict object segmentation masks based on input prompts. These prompts can be a point, bounding box (i.e., a rectangle), mask, or a combination.

The model converts the image into an image embedding (a dense vector representation of the image), which is used to predict masks based on a user prompt.

The `SAM2ImagePredictor` class provides an easy interface to the model. Users can attach the input image to the model using its `set_image` method, which calculates the image embeddings. Then, users can use the `predict` method to share prompts (user inputs) that help with the segmentation mask prediction.

## Set-up

Necessary package and checkpoints' imports and helper functions for displaying points, boxes, and masks.

In [ ]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Download SAM-2 checkpoints (saved versions of the model along with its parameters).
# For the Cyber2A hands-on session, we have already downloaded the checkpoints. Hence, we have commented out the download code below. If you are running this code later, you will need to uncomment the code below to download the checkpoints.

# os.chdir("SAM_checkpoints")
# !sh download_checkpoints.sh
# os.chdir("..")

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# Select the device for computation. We will be using CUDA to run this notebook. Other options are provided for running this notebook in different environments.
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

## Example image 1

Open the first example image, create an object, and display it with grid for estimating point and box coordinates.   

In [ ]:
image = Image.open('data/images/20180917-112527-reduced.jpg')
image = np.array(image.convert("RGB"))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.grid(visible=True)
plt.axis('on')
plt.show()

## Loading the SAM 2 model and configuration

Load the SAM 2 model and predictor. Here, we provide the path to a SAM 2 checkpoint, and it's corresponding configuration YAML file (added during SAM 2 installation).

In [ ]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "SAM_checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)
predictor = SAM2ImagePredictor(sam2_model)

Process the image to produce an image embedding by calling `SAM2ImagePredictor.set_image`. `SAM2ImagePredictor` stores this embedding and will use it for subsequent mask prediction.

In [ ]:
predictor.set_image(image)

### Specifying an object or region using a single point

In this example image, to prompt for the glacier region, let's choose a point on it. 

Points are a type of input to the model. It's represented in (x,y) format and comes with corresponding labels 1 or 0, which are used to represent foreground and background respectively. As we will see later, we can use multiple points as input, but here we use only one. The show_points method displays the selected point using a star icon.

In [ ]:
input_point = np.array([[600, 400]])
input_label = np.array([1])

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()

In [ ]:
# Display the image embedding feature dimension
print(predictor._features["image_embed"].shape, predictor._features["image_embed"][-1].shape)

Predict segmentation mask with `SAM2ImagePredictor.predict`. The model returns segmentation masks, quality predictions for those masks, and low resolution mask logits that can be passed to the next iteration of prediction.

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)
sorted_ind = np.argsort(scores)[::-1] # Sorting the scores in decreasing order
masks = masks[sorted_ind]
scores = scores[sorted_ind]
logits = logits[sorted_ind]

With `multimask_output=True` (the default setting), SAM 2 outputs 3 masks, where `scores` gives the model's own estimation of the quality of these masks. This setting is intended for ambiguous input prompts, and helps the model disambiguate different objects consistent with the prompt. When `False`, it will return a single mask. For ambiguous prompts such as a single point, it is recommended to use `multimask_output=True` even if only a single mask is desired; the best single mask can be chosen by picking the one with the highest score returned in `scores`. This will often result in a better mask.

In [ ]:
masks.shape  # (number_of_masks) x H x W

In [ ]:
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label, borders=True)

## Activity 1: Specifying an object or region using multiple points

We can see that the single input point can be ambiguous, and the model has returned multiple sub-regions within the glacier image. To obtain a single object or region without ambiguity, multiple points can be provided. If available, a mask from a previous iteration can also be supplied to the model to aid in prediction. When specifying a single object with multiple prompts, a single mask can be requested by setting `multimask_output=False`.

In [ ]:
# E.g., input format for specifying two points
# input_point = np.array([[x1, y1], [x2, y2]])
# input_label = np.array([1, 1])

# TODO: In the below piece of code, replace "None" with your two input points. You can specify more points if needed, but please make sure to increase the labels as well. 

input_point = None
input_label = np.array([1, 1])

mask_input = logits[np.argmax(scores), :, :]  # Choose the model's best mask from previous iteration

In [ ]:
masks, scores, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
masks.shape

In [ ]:
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label)

## Example image 2

Open the second example image, create an object, and display it with grid for estimating point and box coordinates.   

In [ ]:
image = Image.open('data/images/20180917-115018-reduced.jpg')
image = np.array(image.convert("RGB"))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.grid(visible=True)
plt.axis('on')
plt.show()

In [ ]:
# Replace image in the predictor with the new image 
predictor.set_image(image)

## Activity 2: Specifying an object or region using multiple points (foreground and background) 
To exclude glacier and just include the glacial discharge, a background point (with label 0) can be supplied.

In [ ]:
# E.g., input format for specifying three points
# input_point = np.array([[x1, y1], [x2, y2], [x3, y3]])
# input_label = np.array([1, 1, 0])

# TODO: In the below piece of code, use two or three input points, of which at least one needs to be a background point (water). You can specify more points if needed, but please make sure to increase the labels as well.
input_point = None
input_label = None

In [ ]:
masks, scores, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label)

## Activity 3: Specifying a specific object with a box

The model can also take a box as input, provided in (x1, y1, x2, y2) format.

In [ ]:
# E.g., input format for specifying a box
# input_box = np.array([x1, y1, x2, y2])

# TODO: In the below piece of code, replace "None" with a box coordinate.
input_box = None

In [ ]:
masks, scores, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, box_coords=input_box)

## Combining points and boxes (Optional)

Points and boxes may be combined, just by including both types of prompts to the predictor. Here this can be used to select glacial discharge region from the bounding box.

In [ ]:
input_box = np.array([400, 150, 640, 400])
input_point = np.array([[550, 350], [550, 225]])
input_label = np.array([0, 0])

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box=input_box,
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, box_coords=input_box, point_coords=input_point, input_labels=input_label)

## Batched prompt inputs (Optional)

`SAM2ImagePredictor` can take multiple input prompts for the same image, using `predict` method. For example, imagine we have several box outputs from an object detector.

In [ ]:
input_boxes = np.array([
    [400, 100, 640, 400],
    [0, 0, 640, 350]
])

In [ ]:
masks, scores, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_boxes,
    multimask_output=False,
)

In [ ]:
masks.shape  # (batch_size) x (num_predicted_masks_per_input) x H x W

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
for mask in masks:
    show_mask(mask.squeeze(0), plt.gca(), random_color=True)
for box in input_boxes:
    show_box(box, plt.gca())
plt.axis('off')
plt.show()

## Automatic MAsk Generation (Optional)

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask
        if borders:
            import cv2
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            # Try to smooth contours
            contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
            cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1)

    ax.imshow(img)

In [ ]:
mask_generator = SAM2AutomaticMaskGenerator(sam2_model)
masks = mask_generator.generate(image)

In [ ]:
print(len(masks))
print(masks[0].keys())

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()